## Preprocessing

### Estrazione frasi
Estraiamo un campione di frasi a partire dai file conllu di Wikipedia italiana elaborata. 

In [35]:
import os
import re
import pandas as pd
from tqdm import tqdm


In [36]:
def get_id(line):
    sent_id = re.sub(r'\D', '', line)
    return sent_id

def get_text(line):
    sent = line[9:].rstrip('\n')
    return sent

def get_sentences(file_conllu):
    sentences = []
    ids = []
    with tqdm(total=sum(1 for _ in open(file_conllu, 'r', encoding='utf-8')), desc=f'Progresso estrazione frasi del file {file_conllu}') as pbar:
        for line in open(file_conllu, 'r', encoding='utf-8'):
            pbar.update(1)  # Aggiorna la barra di avanzamento
            if line.startswith("# sent_id"):
                current_id = get_id(line)  
                ids.append(current_id)
            elif line.startswith("# text"):
                current_sent = get_text(line)
                sentences.append(current_sent)
    return ids, sentences

In [37]:
#nel caso in cui dovessimo itereare sui diversi file nella dir
"""ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"
ds_files = []
for file_name in os.listdir(ds_directory):
    file_path = os.path.join(ds_directory, file_name)
    ds_files.append(file_path)  #ottengo il nome dei diversi file all'interno della directory
print(ds_files)"""

'ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"\nds_files = []\nfor file_name in os.listdir(ds_directory):\n    file_path = os.path.join(ds_directory, file_name)\n    ds_files.append(file_path)  #ottengo il nome dei diversi file all\'interno della directory\nprint(ds_files)'

In [38]:
"""data_df = pd.DataFrame(columns=["id", "text"])
data_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file"""

'data_df = pd.DataFrame(columns=["id", "text"])\ndata_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file'

In [39]:
data_df = pd.read_csv("data/csv/sample_rs42.csv", encoding ="utf-8", index_col=False)

In [40]:
data_df.head()

,sent_id,text,gulpease
0,772665,Il trio non era inizialmente interessato a lav...,52
1,1499173,La ragazza riesce a convincere Lachlan a lasci...,47
2,1346921,"In quanto campione del mondo uscente, egli avr...",34
3,532939,"Scoperto nel 2000, presenta un'orbita caratter...",40
4,363199,Definizione dell'Autorità per l'Energia Elettr...,62


In [41]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   sent_id   1000000 non-null  int64 
 1   text      1000000 non-null  object
 2   gulpease  1000000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 22.9+ MB


In [42]:
samplino = data_df.sample(300) 

In [43]:
samplino.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 540434 to 580271
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sent_id   300 non-null    int64 
 1   text      300 non-null    object
 2   gulpease  300 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 9.4+ KB


### Estrazione degli indici di leggibilità per ogni frase

Utilizziamo l'API _read-it_ per calcolare gli indici di complessità di ogni frase.

In [44]:
import requests
import sys
import json

In [45]:

SERVER_PATH = "http://api.italianlp.it"
#SERVER_PATH = "http://itanlp-gpu.ilc.cnr.it:13000"

def load_document(text):
    try:
        r = requests.post(SERVER_PATH + '/documents/',           # carica il documento nel database del server
                        data={'text': text,                    # durante il caricamento viene eseguita un'analisi linguistica necessaria per calcolare la leggibilità
                            'lang': 'IT',
                            'extra_tasks': ["readability"]     # chiede al server di calcolare anche la leggibilità del docuemnto
                    })
        r.raise_for_status()  
        doc_id = r.json()['id']                           # id del documento nel database del server, che serve per richiedere i risultati delle analisi     
        return doc_id
    except requests.RequestException as e:
        print(f"Errore nel caricamento del documento: {e} \n ---------- Documento: ----------\n{text}")
        with open('frasi_saltate.txt', 'a', encoding='utf-8') as frasi_saltate:
            frasi_saltate.write(f"DOCUMENTO SALTATO: \n{text}")
        return None
    

def get_doc_score(doc_id, text):
    try:
        r = requests.get(SERVER_PATH + '/documents/details/%s' % doc_id)
        r.raise_for_status()
        result = r.json()
        return result
    except requests.RequestException as e:
        print(f"Errore nel recupero dei risultati: {e}\n Documento: {text}: ")
        return None




In [46]:
#facciamo una prova con un testo di poche frasi
text = "Quando sei qui con me, questa stanza non ha più pareti, ma alberi. La vita è difficile quando non sai dove andare a sbattere la testa."


doc_id = load_document(text)
r_score = get_doc_score(doc_id, text)

#vediamo com'è strutturato l'oggetto che viene restituito dall'API
print(r_score)

{'named_entity_executed': False, 'doc_time': '2024-05-22T17:25:43.609487Z', 'readability_executed': True, 'sentences': {'count': 2, 'prev': None, 'data': [{'sentiment_positive_negative_probability': None, 'sentiment_value': None, 'sequence': 1, 'readability_score_all': 61.70159263084, 'sentiment_negative_probability': None, 'sentiment_neutral_probability': None, 'tokens': [{'word': 'Quando', 'ten': None, 'sequence': 1, 'pos': 'CS', 'per': None, 'lemma': 'quando', 'num': None, 'named_entity_instance': None, 'dep_type': 'ROOT', 'dep_parent_sequence': None, 'cpos': 'C', 'gen': None, 'mod': None}, {'word': 'sei', 'ten': None, 'sequence': 2, 'pos': 'N', 'per': None, 'lemma': 'sei', 'num': None, 'named_entity_instance': None, 'dep_type': 'mod', 'dep_parent_sequence': 3, 'cpos': 'N', 'gen': None, 'mod': None}, {'word': 'qui', 'ten': None, 'sequence': 3, 'pos': 'B', 'per': None, 'lemma': 'qui', 'num': None, 'named_entity_instance': None, 'dep_type': 'mod_loc', 'dep_parent_sequence': 4, 'cpos':

In [50]:
#qui iteriamo sui risultati ottenuti per un singolo documento
def get_sen_scores(doc_scores):
    all_sent_rscore = []
    all_sent_text = []
    for i, sent_results in enumerate(doc_scores['sentences']['data']):
        all_sent_rscore.append(sent_results['readability_score_all'])
        all_sent_text.append(sent_results['raw_text'])
    return all_sent_text, all_sent_rscore

#si passa all'api un doc di 1000 frasi per volta.
def readability_extraction(sen_list, x): #x è il numero di frasi che si vogliono calcolare per iterazione 
    all_scores = []
    with open("json_scores5.json", "a") as f:
        f.write("[")
    for i in tqdm(range(0, len(sen_list), x), desc=f"Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a {x} frasi)"):
        doc = "\n".join(sen_list[i:i+x])
        doc_id = load_document(doc)
        r_scores = get_doc_score(doc_id, doc)
        with open("json_scores5.json", "a") as f:
            json.dump(r_scores, f)
            if i + x < len(sen_list):
                f.write(',')  # Aggiungiamo una virgola se non siamo all'ultima iterazione
    with open("json_scores5.json", "a") as f:
        f.write("]")
    #all_scores.append(r_scores)
    

#qui si ottiene la struttura dati finale dove per ogni frase abbiamo lo score ottenuto da read_it
def get_final_data(result_dict):
    text_list = []
    score_list = []
    for doc_scores in result_dict:
        text, score = get_sen_scores(doc_scores)
        text_list = text_list + text
        score_list = score_list + score
    return text_list, score_list


In [51]:
all_results = readability_extraction(samplino["text"].tolist(), 100)

Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a 100 frasi): 100%|██████████| 3/3 [01:01<00:00, 20.35s/it]


In [53]:
all_results = open("json_scores5.json", encoding="utf-8")
results_dict = json.load(all_results)
all_sent, all_scores = get_final_data(results_dict)

### Creazione e valutazione del dataset finale

Si ottiene una struttura dataframe con _id_, _testo e _leggibilità_ per ogni frase. Dopodiché valutiamo le caratteristiche dei dati ottenuti

In [ ]:
#si crea un dataframe con tutte le informazioni che ci interessano
data_set = pd.DataFrame(columns=["id", "text", "readability"])
data_set["text"] = all_sent
data_set["readability"] = all_scores
data_set["id"] = range(1, len(data_set) + 1)
data_set.head()

,id,text,readability
0,1,"La stagione 1989/90 iniziò bene per l'Everton,...",92.466578
1,2,Non è mai stato tradotto in formato digitale d...,81.356359
2,3,Nel dicembre 2012 avviene la richiesta di fall...,42.361172
3,4,Non è chiaro se il termine Dominazioni usato d...,95.701924
4,5,Nochlin si è sposata due volte.,1.908955


In [54]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294 entries, 0 to 301
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           294 non-null    int64  
 1   text         294 non-null    object 
 2   readability  294 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.2+ KB


Si controlla l'ordine dei dati. Da tenere in considerazione il fatto che "all_results" è in forma di lista contenente i dizionari dei risultati dell'API per ogni "documento" (ovvero ogni x frasi). Quindi l'indice utilizzato per gli elementi di "all_results" per controllare l'ordine sarà sempre compreso tra 0 e x. Mentre quello del nostro dataframe sarà l'equivalente + x*y (dove y, è il numero del documento che stiamo prendendo in considerazione)

In [55]:
print(len(results_dict[0]))

36


In [56]:
#ora controlliamo che l'ordine sia corretto...
tupla = data_set[data_set["id"]==35]
da_result = results_dict[0]['sentences']['data'][34] #primo "documento"
print(f"---- Nel dataframe finale ----\n Testo: '{tupla['text'].item()}' - R_score: {tupla['readability'].item()}\n\n") 
print(f"---- Nei risultati ottenuti dall'API ----\n Testo: '{da_result['raw_text']}' - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ----
 Testo: 'Durante il regno di Ḥasan Kuçek, non tutti i Chupanidi restarono leali.' - R_score: 43.5424813612392


---- Nei risultati ottenuti dall'API ----
 Testo: 'la città non veniva servita da tre anni.' - R_score: 30.9012389143344


In [57]:
tupla = data_set[data_set["id"]==49]
da_result = results_dict[1]['sentences']['data'][48] #secondo "documento"
print(f"---- Nel dataframe finale ---- \n Testo: {tupla['text'].item()} - R_score: {tupla['readability'].item()}\n\n") 
print(f"---- Nei risultati ottenuti dall'API ---- \n  Testo: {da_result['raw_text']} - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ---- 
 Testo: Il verdetto è un film del 1948, diretto dal regista Lewis Allen. - R_score: 39.4039916534288


---- Nei risultati ottenuti dall'API ---- 
  Testo: La nazionale di calcio della Polonia (pol. "Reprezentacja Polski w piłce nożnej mężczyzn") è la rappresentativa calcistica della Polonia ed è posta sotto l'egida della federazione calcistica polacca. - R_score: 58.4271426139914


In [58]:
valori_nulli = data_set[data_set["readability"].isnull()]
valori_nulli #vediamo che le frasi troppo corte non restituiscono un valore di leggibilità. 

,id,text,readability


In [59]:
data_set = data_set.dropna()
data_set.to_csv("ds_leggibilita.csv", index=False)